# Bibliotheken

In [1]:
import pandas as pd
import json
import networkx as nx
import re
from sklearn.metrics.pairwise import cosine_similarity

# Daten einlesen

In [61]:
namen = pd.read_excel("namen.xlsx")
namen.index = namen["CouncillorId"]
namen = namen.T
namen = namen[1:]
namen.head()

CouncillorId,CouncillorBioId,CouncillorName,Rat,Fraktion,Kanton,Geburtsdatum,Vereidigungsdatum,nan
3055,4154,Addor Jean-Luc,NR,V,VS,1964-04-22,2015-11-30,4154
2670,3867,Aebi Andreas,NR,V,BE,1958-11-26,2015-11-30,3867
2760,4049,Aebischer Matthias,NR,S,BE,1967-10-18,2015-11-30,4049
2758,4053,Aeschi Thomas,NR,V,ZG,1979-01-13,2015-11-30,4053
2796,4090,Amaudruz Céline,NR,V,GE,1979-03-15,2015-11-30,4090


In [3]:
df = pd.read_excel("parlament_export_session_5019_de_CH.xlsx")
df = df.drop(['Decision','Ja', "Nein", "Enth.", "Entschuldigung gem. Art. 57 Abs. 4", "Hat nicht teilgenommen", "Präsident"], axis=1)
df.head()

,VoteDate,Rat,Kommission,Dept.,AffairId,AffairTitle,VoteRegistrationNumber,VoteMeaningYes,VoteMeaningNo,DivisionText,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
0,2019-09-09 15:26:55,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19203,Accettare l'articolo,Respingere l'articolo,NaN,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
1,2019-09-09 15:27:54,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19204,Accettare il progetto,Respingere il progetto,Votazione sul complesso,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
2,2019-09-09 16:16:12,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19207,Proposta della maggioranza (entrare in materia),Proposta della minoranza Imark (non entrare in...,Entrare in materia,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein,Nein
3,2019-09-09 17:33:42,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19216,Proposta della maggioranza e Consiglio federale,Proposta della minoranza Müller-Altermatt,"Cifra II, Legge sul CO2, art. 3 cpv. 1bis",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja
4,2019-09-09 17:34:46,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19209,Proposta della maggioranza e Consiglio federal...,Proposta della minoranza Girod,"Cifra II, Legge sul CO2, art. 10 cpv. 1bis e 2...",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja


# Index Setzen

In [4]:
df.index = df["AffairTitle"]
df.head()

,VoteDate,Rat,Kommission,Dept.,AffairId,AffairTitle,VoteRegistrationNumber,VoteMeaningYes,VoteMeaningNo,DivisionText,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
AffairTitle,,,,,,,,,,,,,,,,,,,,,
Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,2019-09-09 15:26:55,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19203,Accettare l'articolo,Respingere l'articolo,NaN,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,2019-09-09 15:27:54,NR,"UREK-NR, UREK-SR",UVEK,18.097,Realisierung der zweiten Etappe der 3. Rhoneko...,19204,Accettare il progetto,Respingere il progetto,Votazione sul complesso,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Ja
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 16:16:12,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19207,Proposta della maggioranza (entrare in materia),Proposta della minoranza Imark (non entrare in...,Entrare in materia,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein,Nein
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 17:33:42,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19216,Proposta della maggioranza e Consiglio federale,Proposta della minoranza Müller-Altermatt,"Cifra II, Legge sul CO2, art. 3 cpv. 1bis",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja
"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",2019-09-09 17:34:46,NR,"UREK-NR, UREK-SR",Parl,17.405,Verlängerung der Befristung der Steuererleicht...,19209,Proposta della maggioranza e Consiglio federal...,Proposta della minoranza Girod,"Cifra II, Legge sul CO2, art. 10 cpv. 1bis e 2...",...,Ja,Ja,Nein,Ja,Nein,Nein,Ja,Nein,Ja,Ja


# Tabelle 90 Grad Kippen

In [5]:
df_t = df.T
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
VoteDate,2019-09-09 15:26:55,2019-09-09 15:27:54,2019-09-09 16:16:12,2019-09-09 17:33:42,2019-09-09 17:34:46,2019-09-09 17:36:08,2019-09-09 17:37:27,2019-09-09 17:38:25,2019-09-09 17:39:22,2019-09-09 17:40:54,...,2019-09-27 09:23:53,2019-09-27 09:24:41,2019-09-27 09:44:37,2019-09-27 09:45:36,2019-09-27 09:46:26,2019-09-27 09:47:14,2019-09-27 09:48:23,2019-09-27 09:49:33,2019-09-27 09:50:25,2019-09-27 09:54:11
Rat,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR,...,NR,NR,NR,NR,NR,NR,NR,NR,NR,NR
Kommission,"UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR","UREK-NR, UREK-SR",...,"RK-NR, WBK-SR","RK-NR, RK-SR","WAK-NR, WAK-SR","WBK-NR, SGK-SR","SGK-NR, SGK-SR","UREK-NR, UREK-SR","WBK-NR, SGK-SR","APK-NR, APK-SR","SiK-NR, SiK-SR","SPK-NR, SPK-SR"
Dept.,UVEK,UVEK,Parl,Parl,Parl,Parl,Parl,Parl,Parl,Parl,...,EJPD,EJPD,EFD,EDI,EDI,UVEK,EDI,EJPD,VBS,WBF
AffairId,18.097,18.097,17.405,17.405,17.405,17.405,17.405,17.405,17.405,17.405,...,17.069,18.049,18.05,18.052,18.091,18.095,18.441,19.017,19.022,19.024


# Ersten Infofelder übergehen

In [6]:
df_t = df_t[12:]
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
4154,Enthaltung,Enthaltung,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Enthaltung,Ja,Ja,Ja,Nein,Ja,Nein
3867,Ja,Ja,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Enthaltung,Ja,Ja,Nein
4049,Ja,Ja,Ja,Nein,Nein,Nein,Nein,Nein,Nein,Nein,...,Ja,Nein,Nein,Nein,Ja,Ja,Ja,Ja,Enthaltung,Ja
4053,Ja,Ja,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Nein,Ja,Nein
4090,Hat nicht teilgenommen,Ja,Nein,Ja,Ja,Ja,Ja,Ja,Ja,Ja,...,Ja,Ja,Ja,Ja,Ja,Ja,Ja,Nein,Ja,Nein


# Kategorien durch Zahlen ersetzen

In [7]:
cleanup_dict = {"Ja": 1, "Nein": 0, "Hat nicht teilgenommen": 2, "Enthaltung": 3, "Der Präsident stimmt nicht": 4, "Entschuldigt": 5}

In [8]:
df_t.replace(cleanup_dict, inplace=True)
df_t.head()

AffairTitle,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,Realisierung der zweiten Etappe der 3. Rhonekorrektion. Gesamtkredit,"Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe","Verlängerung der Befristung der Steuererleichterungen für Erdgas, Flüssiggas und biogene Treibstoffe",...,Urheberrechtsgesetz. Änderung,Bundesgesetz über elektronische Identifizierungsdienste,Steuerliche Berücksichtigung der Kinderdrittbetreuungskosten,Für einen vernünftigen Vaterschaftsurlaub - zum Nutzen der ganzen Familie. Volksinitiative,Bundesgesetz über die Familienzulagen. Änderung,Umweltschutzgesetz. Verbot des Inverkehrbringens von illegal geschlagenem Holz,Indirekter Gegenentwurf zur Vaterschaftsurlaubs-Initiative,"Vereinbarung zwischen der Schweiz, Norwegen, Island und Liechtenstein einerseits und der Europäischen Union andererseits zur Beteiligung dieser Staaten an der Europäischen Agentur für das Betriebsmanagement von IT-Grosssystemen. Genehmigung",Armeebotschaft 2019,Beiträge an die Kosten der Kantone für die Kontrolle der Einhaltung der Stellenmeldepflicht. Bundesgesetz
4154,3,3,0,1,1,1,1,1,1,1,...,1,1,1,3,1,1,1,0,1,0
3867,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,3,1,1,0
4049,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,3,1
4053,1,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,1,0
4090,2,1,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,0


# Cosine Similarity berechnen

In [67]:
pd.DataFrame(cosine_similarity([[1,0,1,0],[0,0,0,0],[1,0,1,1]]))

,0,1,2
0,1.000000,0.0,0.816497
1,0.000000,0.0,0.000000
2,0.816497,0.0,1.000000


In [9]:
dists = cosine_similarity(df_t)
dists_named = pd.DataFrame(dists, columns=df_t.index)
dists_named.index = df_t.index
filtered = dists_named[dists_named > 0.8]
filtered = filtered.fillna(0)
filtered

,4154,3867,4049,4053,4090,4180,1135,4155,4184,4058,...,4142,3912,4224,4194,3931,4057,1142,4226,4178,4179
4154,1.000000,0.000000,0.000000,0.828215,0.814113,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.839167,0.000000
3867,0.000000,1.000000,0.000000,0.805108,0.814318,0.000000,0.000000,0.801022,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.822673,0.000000,0.851318,0.828375
4049,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.804290,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.826541,0.000000,0.000000
4053,0.828215,0.805108,0.000000,1.000000,0.845575,0.000000,0.000000,0.827821,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.849730,0.000000,0.937124,0.854352
4090,0.814113,0.814318,0.000000,0.845575,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.818637,0.000000,0.891853,0.863060
4180,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
1135,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.821141,0.000000
4155,0.000000,0.801022,0.000000,0.827821,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.801228,0.000000,0.864220,0.829389
4184,0.000000,0.000000,0.804290,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.808115,...,0.000000,0.0,0.841807,0.0,0.0,0.0,0.000000,0.859272,0.000000,0.000000
4058,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.808115,1.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.804896,0.000000,0.000000


# Als Netzwerk exportieren

In [46]:
namen_dict = dict(zip(namen["CouncillorBioId"],namen["CouncillorName"]))
fract_dict = dict(zip(namen["CouncillorBioId"],namen["Fraktion"]))
name_mappings = {}
fract_mappings = {}
i = 0
for id in df_t.index:
    name_mappings[i] = namen_dict[id]
    fract_mappings[i] = fract_dict[id]
    i += 1

In [59]:
G=nx.from_numpy_matrix(filtered.values)
nx.set_node_attributes(G, values = fract_mappings, name='fraktion')
H = nx.relabel_nodes(G, name_mappings)
nx.write_gexf(H,"Politics.gexf")